In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('../data/all_data_label_merge.csv')
# df = df.loc[:4999]

In [3]:
def get_freq(DF):
    words_list = []
    try:
        for idx, words in enumerate(DF.ngram):
            words_list.extend(words.replace('[','').replace(']','').replace('\'','').replace(' ','').split(','))
    except:
        print(idx)
        pass
    freq_counter = Counter(words_list)
    
    return freq_counter

In [4]:
up_df = df[df['label'] == 'up']
down_df = df[df['label'] == 'down']

up_counter = get_freq(up_df)
down_counter = get_freq(down_df)

total_ngram = set(get_freq(df).keys())

In [5]:
df_score = pd.DataFrame(columns=['ngram','up_count','down_count'])
df_score.ngram = list(total_ngram)

for row, ngram in tqdm(enumerate(df_score.ngram)):
    df_score.loc[row,'up_count'] = up_counter[ngram]
    df_score.loc[row,'down_count'] = down_counter[ngram]

df_score['total_count'] = df_score['up_count'] + df_score['down_count']

0it [00:00, ?it/s]

In [6]:
df_score.drop(df_score.index[df_score[df_score.total_count<15].index], inplace=True)

In [7]:
df_score.sort_values(by='total_count', ascending=True)

,ngram,up_count,down_count,total_count
1203,금리/NNG;상승/NNG;시장/NNG;약세/NNG,8,7,15
26227,경기/NNG;판단/NNG;개선/NNG,8,7,15
5080,악화/NNG;감소/NNG,7,8,15
30565,신용등급/NNG;강등/NNG;등급/NNG;전망/NNG;부정적/VAX,6,9,15
27508,잠재/NNG;불확실성/NNG,10,5,15
...,...,...,...,...
24901,금리/NNG;상승/NNG,2737,2271,5008
3093,금리/NNG;인하/NNG,2954,5184,8138
7970,fed/NNG;금리/NNG;인상/NNG,5899,5076,10975
0,,5932,6727,12659


In [8]:
# p_hawkish = sum(df_score.up_count)/sum(df_score.total_count)
# p_dovish = sum(df_score.down_count)/sum(df_score.total_count)

n_hawkish = sum(df_score.up_count)
n_dovish = sum(df_score.down_count)

p_hawkish = df_score.up_count/n_hawkish
p_dovish = df_score.down_count/n_dovish

df_score['polar_score'] = p_hawkish.map(lambda x: np.float(x))/p_dovish.map(lambda x: np.float(x))
df_score['polarity'] = np.where(df_score['polar_score'] > (1.3/1), 'Hawkish', np.where(df_score['polar_score'] < (1/1.3), 'Dovish','nothing'))

/var/folders/85/178rxd5d5b31szqc8b3mznd40000gn/T/ipykernel_2694/4211945582.py:10: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_score['polar_score'] = p_hawkish.map(lambda x: np.float(x))/p_dovish.map(lambda x: np.float(x))


In [9]:
df_score.reset_index(drop=True, inplace=True)

In [10]:
hawk_dic = df_score[df_score.polarity == 'Hawkish'].reset_index(drop=True)
dove_dic = df_score[df_score.polarity == 'Dovish'].reset_index(drop=True)

hawk_dic.to_csv('../data/hawk_dic.csv',index = False)
dove_dic.to_csv('../data/dove_dic.csv',index = False)